# Training
on entraine le réseau de neurones pour les hyperparamètres

In [1]:
# importation
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
import cv2 as cv
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
# local
from MyResNet.myfunc import Physics
from MyResNet.myfunc import MyMatmul
from MyResNet.model import MyModel

### Paramètres physiques

In [2]:
a = 1
p = 1
nx = 200
tensor_list = Physics(a,p,nx)
Tt          = MyMatmul(tensor_list[1].T)
mass        = 1
U           = torch.FloatTensor(np.ones(nx))

In [3]:
def CreateLoader(folder):
    """
    According to the mode, creates the appropriate loader 
    for the training and validation sets.
    """
    dfx     = pd.read_csv(folder+'/'+'data_lisse.csv', sep=',',header=None)
    dfy     = pd.read_csv(folder+'/'+'data_blurred.csv', sep=',',header=None)
    _,nx    = dfx.shape
    #
    x_tensor= torch.FloatTensor(dfx.values[:50]).view(-1,1,nx)
    y_tensor= torch.FloatTensor(dfy.values[:50]).view(-1,1,nx)
    #
    dataset = TensorDataset(x_tensor, y_tensor)
    l = len(dataset)
    m = 2*l//3
    train_dataset, val_dataset = random_split(dataset, [m, l-m])
    #
    train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True)
    val_loader   = DataLoader(val_dataset, batch_size=1, shuffle=False)
    return train_loader, val_loader

In [4]:
folder = './MyResNet/Dataset'
t, v = CreateLoader(folder)

### Model

In [5]:
mymodel = MyModel(tensor_list,mass,U,20)

In [6]:
loss_fn = nn.MSELoss(reduction='mean')

### Training

In [7]:
def train(model,nb_epochs):
    """
    Trains iRestNet.
    """      
    # to store results
    loss_epochs  =  np.zeros(nb_epochs)
    loss_train   =  np.zeros(nb_epochs)
    loss_val     =  np.zeros(nb_epochs)
    loss_min_val =  float('Inf')
    # defines the optimizer
    lr_i        = 0.01
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad,model.parameters()),lr=lr_i)

    #==========================================================================================================
    # trains for several epochs
    for epoch in range(0,nb_epochs): 
        # sets training mode
        model.train()
        # modifies learning rate
        if epoch>0:
            lr_i      = lr_i*0.9 
            optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad,self.parameters()), lr=lr_i)
        # goes through all minibatches
        for i,minibatch in enumerate(t):
            [x_true, x_blurred] = minibatch    # get the minibatch
            x_true    = Variable(x_true,requires_grad=False)
            x_init    = Variable(x_blurred,requires_grad=False)
            # ATTENTION : on ne calcule pas le gradient en fonction de la deuxième sortie (à réfléchir)
            Ttx_init  = Tt(x_init).detach()     # do not compute gradient
            x_pred    = model(x_init,Ttx_init) 
                    
            # Computes and prints loss
            loss                = loss_fn(x_pred, x_true)
            loss_epochs[epoch] += torch.Tensor.item(loss)
                    
            # sets the gradients to zero, performs a backward pass, and updates the weights.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
                
            
        #==========================================================================================================
        # training is finished
        print('-----------------------------------------------------------------')
        print('Training is done.')
        print('-----------------------------------------------------------------')

    

In [8]:
train(mymodel,20)

RuntimeError: Subtraction, the `-` operator, with a bool tensor is not supported. If you are trying to invert a mask, use the `~` or `logical_not()` operator instead.